In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join
from oaklib import OntologyResource
from oaklib.implementations import SqlImplementation

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"

In [ ]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

# Use OAK to get 'object_label'
ontology_resource = OntologyResource(slug='../ontology/tmp/merged.db', local=True)
oi = SqlImplementation(ontology_resource)

In [ ]:
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
"ICD10CM", "MONDO"
condition_1 = msdf_mondo.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_mondo.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_mondo.df['object_id'].str.contains("MONDO")
condition_4 = msdf_mondo.df['subject_id'].str.contains("ICD10CM")
msdf_mondo.df = msdf_mondo.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))
msdf_mondo.df.head()

In [ ]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = msdf_lex.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_lex.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_lex.df['object_id'].str.contains("MONDO")
condition_4 = msdf_lex.df['subject_id'].str.contains("ICD10CM")
msdf_lex.df = msdf_lex.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_lex.df


In [ ]:
comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
comparison_df = comparison_ms_diff.combined_dataframe
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
comparison_df.head()

In [ ]:
mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
unmapped_df = comparison_df[
    (comparison_df['comment'].str.contains("|".join(mappings)))
]

unmapped_df.head()

In [ ]:
# Let reviewers check if this makes sense or no.
unmapped_lex_exact = unmapped_df[(unmapped_df['comment'] == 'LEXMATCH') & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_lex_exact.to_csv(join(dir_name, "unmapped_lex_exact.tsv"), sep='\t', index = False)
unmapped_lex_exact.head()

In [ ]:
# Inspect why these are missing from SSSOM mappings
# unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))
unmapped_mondo_exact = unmapped_df[(unmapped_df['comment'] == 'MONDO_MAPPINGS')& (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
unmapped_mondo_exact.head()

In [ ]:
combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=msdf_lex.prefix_map, metadata=msdf_lex.metadata)
combined_msdf.df.head()

In [ ]:
%%time
df_dict = split_dataframe(combined_msdf)

In [ ]:
mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
mondo_icd_list

In [ ]:
for match in mondo_icd_list:
    fn = match + ".tsv"
    df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [ ]:
df_dict['mondo_exactmatch_icd10cm'].df